### Configs and Libs

In [1]:
SCOPETYPE = 'CWNANO'
PLATFORM = 'CWNANO'
SS_VER = 'SS_VER_1_1'
CRYPTO_TARGET = 'NONE'

### Creating a New Directory

The following bash block will create a new directory and copy the `simpleserial-base` code into it:

In [2]:
'''%%bash
cd ../../../hardware/victims/firmware/
mkdir -p ops_with_key && cp -r simpleserial-base/* $_
cd ops_with_key'''

'%%bash\ncd ../../../hardware/victims/firmware/\nmkdir -p ops_with_key && cp -r simpleserial-base/* $_\ncd ops_with_key'

### Connecting to Hardware
Quand j'éxecute cette ligne le scope est defini

In [3]:
%run "../../Setup_Scripts/Setup_Generic.ipynb"

INFO: Found ChipWhisperer😍


In [4]:
print(target)
print(scope)

SimpleSerial Settings =
	output_len             = 16
	baud                   = 38400
	simpleserial_last_read = 
	simpleserial_last_sent = 
	
ChipWhisperer Nano Device
fw_version = 
    major = 0
    minor = 65
    debug = 0
io = 
    tio1         = None
    tio2         = None
    tio3         = None
    tio4         = high_z
    pdid         = True
    pdic         = False
    nrst         = True
    clkout       = 7500000.0
    cdc_settings = bytearray(b'\x01\x00\x00\x00')
adc = 
    clk_src  = int
    clk_freq = 7500000.0
    samples  = 5000
glitch = 
    repeat     = 0
    ext_offset = 0



### Building Firmware

Rerun this every time you make changes to your code:

In [5]:
%%bash -s "$PLATFORM" "$SS_VER" "$CRYPTO_TARGET"
cd ../../../hardware/victims/firmware/ops_with_key
make PLATFORM=$1 SS_VER=$2 CRYPTO_TARGET=$3 -j

SS_VER set to SS_VER_1_1
SS_VER set to SS_VER_1_1
SS_VER set to SS_VER_1_1
SS_VER set to SS_VER_1_1
make[1]: '.dep' is up to date.
SS_VER set to SS_VER_1_1
SS_VER set to SS_VER_1_1
.
Welcome to another exciting ChipWhisperer target build!!
arm-none-eabi-gcc.exe (GNU Arm Embedded Toolchain 10-2020-q4-major) 10.2.1 20201103 (release)
Copyright (C) 2020 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

.
Assembling: .././hal/stm32f0/stm32f0_startup.S
arm-none-eabi-gcc -c -mcpu=cortex-m0 -I. -x assembler-with-cpp -mthumb -mfloat-abi=soft -ffunction-sections -DF_CPU=7372800 -Wa,-gstabs,-adhlns=objdir-CWNANO/stm32f0_startup.lst -I.././simpleserial/ -I.././simpleserial/ -I.././hal -I.././hal/stm32f0 -I.././hal/stm32f0/CMSIS -I.././hal/stm32f0/CMSIS/core -I.././hal/stm32f0/CMSIS/device -I.././hal/stm32f0/Legacy -I.././crypto/ .././hal/stm32f0/stm32f0_startup.S -o

### Uploading Firmware


In [6]:
cw.program_target(scope, prog, "../../../hardware/victims/firmware/ops_with_key/simpleserial-base-{}.hex".format(PLATFORM))

Detected known STMF32: STM32F03xx4/03xx6
Extended erase (0x44), this can take ten seconds or more
Attempting to program 4443 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 4443 bytes


## Trace Capture

In [7]:
#validado
from tqdm.notebook import trange
import numpy as np
import time

ktp = cw.ktp.Basic()
textin_array = []
textout_array = []

N = 1000
for i in trange(N, desc='Capturing data'):
    key, text = ktp.next() 
    trace = cw.capture_trace(scope, target, text)

    if not trace:
        continue

    textout_array.append(trace.textout)
    textin_array.append(trace.textin)

Capturing data:   0%|          | 0/10 [00:00<?, ?it/s]

In [8]:
textout_array

[bytearray(b'\x84\xa2ID\x12\x18X\xb5\xee^\xe6\xa8\xf9D\x90\xba'),
 bytearray(b'%\xff\xe2\xd9\xf84\xbf|h\x00\xf4ov\xcf\x14\x87'),
 bytearray(b'oM\xc6x\xa9\x8eV:\x1e\x02\xe2\xd7\xc4\xb2z\xe7'),
 bytearray(b'\xb0Np*M\xba\xae\xb5h:+\xa3\x9d\x8e\xf5\x13'),
 bytearray(b'[\x99\xec\x10\xe9>\x0e\x9f\xbeVLyp\xef\xbam'),
 bytearray(b'n@8\x12\xed\x10:[\xdeS\xa1g\x03P\xf7\xd3'),
 bytearray(b'\xc9\xf1\xb7{\xe8\x0c~9{\x80\x82y\x96\xc7\xdcO'),
 bytearray(b'-\xee\xd4\xe8\xea\xc4:\xecq\xbe\x82&\xa59\xd2p'),
 bytearray(b'g\xf4\x9c2\xb7\xc8p\x14\x01\xa3\xd5\x89\x8d\xd7\x8b\xfb'),
 bytearray(b'8\xd8_\x92CE\x05\x96\x18\xb4\xff&\x9cZf\\')]

In [9]:
textin_array

[CWbytearray(b'c6 a2 49 44 12 18 58 b5 ee 5e e6 a8 f9 44 90 ba'),
 CWbytearray(b'67 ff e2 d9 f8 34 bf 7c 68 00 f4 6f 76 cf 14 87'),
 CWbytearray(b'b1 4d c6 78 a9 8e 56 3a 1e 02 e2 d7 c4 b2 7a e7'),
 CWbytearray(b'f2 4e 70 2a 4d ba ae b5 68 3a 2b a3 9d 8e f5 13'),
 CWbytearray(b'9d 99 ec 10 e9 3e 0e 9f be 56 4c 79 70 ef ba 6d'),
 CWbytearray(b'b0 40 38 12 ed 10 3a 5b de 53 a1 67 03 50 f7 d3'),
 CWbytearray(b'0b f1 b7 7b e8 0c 7e 39 7b 80 82 79 96 c7 dc 4f'),
 CWbytearray(b'6f ee d4 e8 ea c4 3a ec 71 be 82 26 a5 39 d2 70'),
 CWbytearray(b'a9 f4 9c 32 b7 c8 70 14 01 a3 d5 89 8d d7 8b fb'),
 CWbytearray(b'7a d8 5f 92 43 45 05 96 18 b4 ff 26 9c 5a 66 5c')]

In [13]:
 for j in range (3) : 
    print("\n");
    for i in range (16) : 
        print(hex(textout_array[j][i]))




0xc6
0xa2
0x49
0x44
0x12
0x18
0x58
0xb5
0xee
0x5e
0xe6
0xa8
0xf9
0x44
0x90
0xba


0x67
0xff
0xe2
0xd9
0xf8
0x34
0xbf
0x7c
0x68
0x0
0xf4
0x6f
0x76
0xcf
0x14
0x87


0xb1
0x4d
0xc6
0x78
0xa9
0x8e
0x56
0x3a
0x1e
0x2
0xe2
0xd7
0xc4
0xb2
0x7a
0xe7


0xf2
0x4e
0x70
0x2a
0x4d
0xba
0xae
0xb5
0x68
0x3a
0x2b
0xa3
0x9d
0x8e
0xf5
0x13


0x9d
0x99
0xec
0x10
0xe9
0x3e
0xe
0x9f
0xbe
0x56
0x4c
0x79
0x70
0xef
0xba
0x6d


0xb0
0x40
0x38
0x12
0xed
0x10
0x3a
0x5b
0xde
0x53
0xa1
0x67
0x3
0x50
0xf7
0xd3


0xb
0xf1
0xb7
0x7b
0xe8
0xc
0x7e
0x39
0x7b
0x80
0x82
0x79
0x96
0xc7
0xdc
0x4f


0x6f
0xee
0xd4
0xe8
0xea
0xc4
0x3a
0xec
0x71
0xbe
0x82
0x26
0xa5
0x39
0xd2
0x70


0xa9
0xf4
0x9c
0x32
0xb7
0xc8
0x70
0x14
0x1
0xa3
0xd5
0x89
0x8d
0xd7
0x8b
0xfb


0x7a
0xd8
0x5f
0x92
0x43
0x45
0x5
0x96
0x18
0xb4
0xff
0x26
0x9c
0x5a
0x66
0x5c


In [15]:
assert(len(textout_array) == N)
assert(len(textin_array) == N)


In [12]:
scope.dis()
target.dis()